# Transfer learning   
### Based on:   
https://medium.com/towards-data-science/transfer-learning-using-keras-d804b2e04ef8

In [1]:
import sys
import numpy as np
import h5py
import copy
import math
import pandas as pd
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.utils.np_utils import to_categorical
import keras

In [ ]:
print('python = {0}'.format(sys.version))
print('numpy = {0}'.format(np.__version__))
print('pandas = {0}'.format(pd.__version__))
print('keras = {0}'.format(keras.__version__))

## Read data. Make train and test sets

In [ ]:
def readScan(scanName,var_name):
    data=h5py.File(scanName,'r')
    Xscans=data.get(var_name)
    X=copy.copy(Xscans.value)
    X1=np.rollaxis(X,0,start=3)
    return X1

In [ ]:
n_echoes=100
skip_rows=3 # the first 50 description files have a different format 

X_rest=[]
X_stress=[]
X_reserve=[]
y=[]

for idx,i in enumerate(np.arange(n_echoes)+1):
    scan_rest='../stress_rest_reserve_100/rest{0}.mat'.format(i)
    X_rest.append(readScan(scan_rest,'rest'))
    scan_stress='../stress_rest_reserve_100/stress{0}.mat'.format(i)
    X_stress.append(readScan(scan_stress,'stress'))
    scan_reserve='../stress_rest_reserve_100/reserve{0}.mat'.format(i)
    X_reserve.append(readScan(scan_reserve,'reserve'))
    # the first 50 description files have a different format 
    if i==51:
        skip_rows=0
    labels=pd.read_csv('../Data100/PD_ANN0%03d.csv' % (i), skiprows=skip_rows)
    # Criteria to determine whether a polar map does not have risk = 0 or has risk = 1
    LAD=labels['Reserve mean'][0]
    LCX=labels['Reserve mean'][1]
    RCA=labels['Reserve mean'][2]
    y_temp=[0 if LAD > 2 and LCX > 2 and RCA > 2 else 1]
    y.append(y_temp[0])

In [ ]:
from collections import deque
# Shuffle (permute) the scans
rand_samples=random.sample(range(0,n_echoes),n_echoes)
# we keep ids of the patients
ids100=np.arange(100)+1
ids=[ids100[i] for i in rand_samples]
X_reserve_shuffle=[X_reserve[i] for i in rand_samples]
# Make a deque object and rotate x times n-folds times
X_=deque(X_reserve_shuffle)
y_=deque(y);ids_=deque(ids)
X_.rotate(20);y_.rotate(20);ids_.rotate(20)
X_reserve_0=list(X_)
y_0=to_categorical(list(y_))
ids0=list(ids_)
X_.rotate(20);y_.rotate(20);ids_.rotate(20)
X_reserve_1=list(X_)
y_1=to_categorical(list(y_))
ids1=list(ids_)
X_.rotate(20);y_.rotate(20);ids_.rotate(20)
X_reserve_2=list(X_)
y_2=to_categorical(list(y_))
ids2=list(ids_)
X_.rotate(20);y_.rotate(20);ids_.rotate(20)
X_reserve_3=list(X_)
y_3=to_categorical(list(y_))
ids3=list(ids_)
X_.rotate(20);y_.rotate(20);ids_.rotate(20)
X_reserve_4=list(X_)
y_4=to_categorical(list(y_))
ids4=list(ids_)

In [ ]:
np.sum(X_reserve_shuffle[0])

In [ ]:
print(np.sum(X_reserve_0[20]),np.sum(X_reserve_1[40]),
      np.sum(X_reserve_2[60]),np.sum(X_reserve_3[80]),
      np.sum(X_reserve_4[0]))

In [ ]:
print(ids0[0:5])
print(ids1[20:25])
print(ids2[40:45])
print(ids3[60:65])
print(ids4[80:85])

In [ ]:
# # permute the samples 5 times
# X_reserve_5=[]
# y_5=[]
# for i in np.arange(5):
#     rand_samples=random.sample(range(0,n_echoes),n_echoes)
#     X_reserve_=[X_reserve[i] for i in rand_samples]
#     y_=[y[i] for i in rand_samples]
#     X_reserve_5.append(np.array(X_reserve_))
#     y_5.append(to_categorical(np.array(y_)))

---

## Add padding to match inception shape

In [ ]:
## size for VGG = 224
## size for inception = 299

In [ ]:
def match_base_model_size(X,network='VGG'):
    """
    Function does not generalize, made only for specific dimensions size
    """
    pixels_base_model=[224 if network=='VGG' else 299]
    ## For VGG
    if int(pixels_base_model[0])-np.shape(X)[1]==1:
        print('VGG')
        pixels_to_add=1
        padding=((0,pixels_to_add),(0,pixels_to_add),(0,0))
        X=[np.pad(i,padding,mode='constant', constant_values=0) for i in X]
    ## For Inception
    elif int((pixels_base_model[0])-np.shape(X)[1])>=2:
        pixels_to_add=int(((pixels_base_model[0])-np.shape(X)[1])/2)
        print('Inception')
        padding=((pixels_to_add,pixels_to_add),(pixels_to_add,pixels_to_add),(0,0))
        X=[np.pad(i,padding,mode='constant', constant_values=0) for i in X]
    else:
        print('check dimensions') 
    return(np.array(X))

In [ ]:
X_reserve_0=match_base_model_size(X_reserve_0)
X_reserve_1=match_base_model_size(X_reserve_1)
X_reserve_2=match_base_model_size(X_reserve_2)
X_reserve_3=match_base_model_size(X_reserve_3)
X_reserve_4=match_base_model_size(X_reserve_4)

In [ ]:
# X_rest=match_base_model_size(X_rest,'VGG')
# X_stress=match_base_model_size(X_stress,'VGG')
# X_reserve=match_base_model_size(X_reserve,'VGG')

In [ ]:
np.shape(X_reserve_0)

In [ ]:
plt.pcolor(X_reserve_0[5][:,:,1])
print(np.shape(X_reserve_0[5][:,:,1]))
plt.figure()
plt.pcolor(X_reserve_1[25][:,:,1])

# Get pretrained model

In [ ]:
# base_model = applications.InceptionV3(weights='imagenet', include_top=False)

In [ ]:
# base_model = applications.VGG19(weights = "imagenet", include_top=False)

In [ ]:
#https://machinelearningmahttps://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5stery.com/save-load-keras-deep-learning-models/
# serialize model to JSON
# model_json = base_model.to_json()
# with open("model_InceptionV3.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# base_model.save_weights("model_InceptionV3.h5")
# print("Saved model to disk")

In [ ]:
#https://machinelearningmastery.com/save-load-keras-deep-learning-models/
# load json and create model
from keras.models import model_from_json
json_file = open('model_VGG19.json', 'r')
#json_file = open('model_InceptionV3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
base_model = model_from_json(loaded_model_json)
# load weights into new model
base_model.load_weights("model_VGG19.h5")
#base_model.load_weights("model_InceptionV3.h5")
print("Loaded model from disk")

---

# Functions   
https://github.com/DeepLearningSandbox/DeepLearningSandbox/blob/master/transfer_learning/fine-tune.py

In [ ]:
# https://deeplearningsandbox.com/how-to-use-transfer-learning-and-fine-tuning-in-keras-and-tensorflow-to-build-an-image-recognition-94b0b02444f2
def add_new_last_layer(base_model, nb_classes):
  """Add last layer to the convnet
  Args:
    base_model: keras model excluding top
    nb_classes: # of classes
  Returns:
    new keras model with last layer
  """
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(FC_SIZE, activation='relu')(x) 
  predictions = Dense(nb_classes, activation='softmax')(x) 
  model = Model(inputs=base_model.input, outputs=predictions)
  return model

In [ ]:
def setup_to_transfer_learn(model, base_model):
  """Freeze all layers and compile the model"""
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer='Adam',    
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

In [ ]:
def setup_to_finetune(model,learning_rate):
   """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top 
      layers.
   note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in 
         the inceptionv3 architecture
   Args:
     model: keras model
   """
   for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
      layer.trainable = False
   for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
      layer.trainable = True
   model.compile(optimizer=SGD(lr=learning_rate, momentum=0.9),   
                 loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
np.shape(X_reserve_0)

---

In [ ]:
nb_classes=2
FC_SIZE = 1024
NB_EPOCHS = 20
BAT_SIZE = 32
NB_IV3_LAYERS_TO_FREEZE = 172

In [ ]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 7.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
# learning schedule callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [ ]:
# setup model
model = add_new_last_layer(base_model, nb_classes)

In [ ]:
# transfer learning
setup_to_transfer_learn(model, base_model)

In [ ]:
history_tl=model.fit(X_reserve_0,y_0,batch_size=BAT_SIZE,
                  epochs=NB_EPOCHS,shuffle=True,
                    validation_split=.2,
                    callbacks=callbacks_list,
                    verbose=2)

In [ ]:
trans_learn_XXX=pd.DataFrame(np.vstack((history_tl.history['acc'],
                             history_tl.history['val_acc'],
                             history_tl.history['loss'],
                             history_tl.history['val_loss'])).T,
                             columns=['acc','val_acc','loss','val_loss',])

In [ ]:
trans_learn_XXX['base_model']='Inception'
trans_learn_XXX['input_image']='Xreserve_0'
trans_learn_XXX['initial_lr']='0.0001'

In [ ]:
trans_learn_XXX.to_csv('Results/trans_learn_0XX_reserve_0.csv',index=False)

---

In [ ]:
y_pred=model.predict(X_reserve_five[0][80:])

In [ ]:
for a,b in zip(y_5[0][80:],y_pred):
    print(a[0],b[0])

In [ ]:
y_=[i[0] for i in y_pred]

In [ ]:
colors=['#FF0000','#00FF00']
a=[colors[int(i[0])] for i in y_5[0][80:]]

In [ ]:
x_=np.ones([1,20])+np.random.rand(1,20)/10

In [ ]:
plt.scatter(x_,y_,s=50, c=a, edgecolor='k')

In [ ]:
a

In [ ]:
for i in 


In [ ]:
N, bins, patches =plt.hist(np.asmatrix(y_pred)[:,0],bins=10,width=.005)
jet = plt.get_cmap('jet', 2)
for i in range(len(patches)):
    patches[i].set_facecolor(jet(int(y_5[0][80:][i][1])))

In [ ]:
for i in range(len(patches)):
    patches[i].set_facecolor(jet(int(y_5[0][80:][i][1])))

---

In [ ]:
plt.plot(history_tl.history['acc'])
plt.plot(history_tl.history['val_acc'],'g')
plt.ylim([0.4,1])
plt.title('ACC: VGG, Adam, lr=0.000001, step decay')
plt.figure()
plt.plot(history_tl.history['loss'])
plt.plot(history_tl.history['val_loss'],'g')
plt.title('Loss: VGG, Adam, lr=0.000001, step decay')
plt.legend(['VGG, X_rest'])

In [ ]:
plt.plot(history_tl.history['acc'])
plt.plot(history_tl.history['val_acc'],'g')
plt.ylim([0.4,1])
plt.title('ACC: VGG, Adam, lr=0.00001, step decay')
plt.figure()
plt.plot(history_tl.history['loss'])
plt.plot(history_tl.history['val_loss'],'g')
plt.title('Loss: VGG, Adam, lr=0.00001, step decay')

In [ ]:
plt.plot(history_tl.history['loss'])
plt.plot(history_tl.history['val_loss'],'g')

In [ ]:
learning_rate=k.get_value(model.optimizer.lr)

In [ ]:
 # fine-tuning
setup_to_finetune(model,learning_rate)

In [ ]:
history_ft=model.fit(X,y,batch_size=BAT_SIZE,
                  epochs=5,shuffle=True,
                    validation_split=.2)

In [ ]:
def plot_training(history_tl,history_ft):
  acc = history_tl.history['acc']+history_ft.history['acc']
  val_acc = history_tl.history['val_acc']+history_ft.history['val_acc']
  loss = history_tl.history['loss']+history_ft.history['loss']
  val_loss = history_tl.history['val_loss']+history_ft.history['val_loss']
  epochs = range(len(acc))
  epochs_tl=len(history_tl.history['acc'])

  plt.plot(epochs, acc, 'b')
  plt.plot(epochs, val_acc, 'g')
  plt.title('Training and validation accuracy')
  plt.vlines(epochs_tl,0,1,linestyle='--')

  plt.figure()
  plt.plot(epochs, loss, 'b')
  plt.plot(epochs, val_loss, 'g')
  plt.title('Training and validation loss')
plt.show()

In [ ]:
plot_training(history_tl,history_ft)

In [ ]:
np.sum(y[80:],0)

In [ ]:
np.sum(y,0)